In [86]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np

Fuer alle 10 Teile ausfuehren

In [87]:
# import

infile = open('vacay_preprocessed9.pkl','rb')
df = pickle.load(infile)
infile.close()

df['temp_ueb'] = df['UebungsID'].map(lambda x: str(x)[:-2])

path = '../../../../../../Desktop/Orthografietrainer2020/pickle/'
infile = open(path+'sitzungssummary.pkl','rb')
sitzungen = pickle.load(infile)
infile.close()
sitzungen = sitzungen[['ID','beendet']]

sitzungen.columns = ['temp_ueb', 'beendet']
sitzungen['temp_ueb'] = sitzungen['temp_ueb'].astype(str)

df_new = pd.merge(df, sitzungen, on='temp_ueb')
df_new = df_new.drop(columns=['Loesungsnr'])
df_new = df_new.drop_duplicates()


In [88]:
luecke_0 = df_new[df_new.luecke == 0]

luecke_0['testpos_prue'] = np.where(luecke_0['Testposition']== 'pruefung', 1, 0)
luecke_0['testpos_train'] = np.where(luecke_0['Testposition']== 'training', 1, 0)
luecke_0['testpos_vers'] = np.where(luecke_0['Testposition']== 'version', 1, 0)
luecke_0_grouped = luecke_0.groupby('UebungsID').agg({'Erfolg': ['count', 'sum'], 'MehrfachFalsch': ['sum'], 'luecke': ['first'], 'sessionNr':['first'], 'vorher_abgebrochen':['first'],'temp_ueb':['first'], 'beendet' : ['first'], 'testpos_prue':['sum'], 'testpos_train':['sum'], 'testpos_vers':['sum']})

def f(row):
    if row.testpos_train['sum'] < 10:
        val = 0
    elif  row.testpos_train['sum'] >= 10:
        if (row.testpos_train['sum'] == 10) and (row.Erfolg['sum'] == 10):
            val = 1
        elif ((row.Erfolg['count']-row.testpos_prue['sum']) == row.Erfolg['sum']) and ((row.testpos_vers['sum']/2+10) ==  row.testpos_train['sum']): #standardweg für Fehler im Training
            val = 1
        elif ((row.Erfolg['count']-row.Erfolg['sum']) == row.testpos_prue['sum']):
            val = 1
        elif ((row.Erfolg['count']-row.Erfolg['sum']+1) == row.testpos_prue['sum']):
            val = 1
        elif (row.testpos_vers['sum']>0) and (row.testpos_prue['sum']==0):
            val = 0
        else:
            val = 3
    else:
        val = 3
    return val

luecke_0_grouped['y'] = luecke_0_grouped.apply(f, axis=1)

<ipython-input-88-97bc509b50d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  luecke_0['testpos_prue'] = np.where(luecke_0['Testposition']== 'pruefung', 1, 0)
<ipython-input-88-97bc509b50d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  luecke_0['testpos_train'] = np.where(luecke_0['Testposition']== 'training', 1, 0)
<ipython-input-88-97bc509b50d4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [89]:
luecke_1 = df_new[df_new.luecke == 1]

luecke_1['testpos_prue'] = np.where(luecke_1['Testposition']== 'pruefung', 1, 0)
luecke_1['testpos_train'] = np.where(luecke_1['Testposition']== 'training', 1, 0)
luecke_1['testpos_vers'] = np.where(luecke_1['Testposition']== 'version', 1, 0)

max_session = luecke_1.groupby('temp_ueb')['sessionNr'].max()
max_session=max_session.reset_index()
max_session.columns = ['temp_ueb', 'maxSession']

luecke_1 = pd.merge(luecke_1, max_session, on='temp_ueb')
luecke_1_grouped = luecke_1.groupby('UebungsID').agg({'Erfolg': ['count', 'sum'], 'MehrfachFalsch': ['sum'], 'luecke': ['first'], 'sessionNr':['first'], 'maxSession':['first'], 'vorher_abgebrochen':['first'],'temp_ueb':['first'], 'beendet' : ['first'], 'testpos_prue':['sum'], 'testpos_train':['sum'], 'testpos_vers':['sum']})
luecke_1_grouped_full = luecke_1.groupby('temp_ueb').agg({'Erfolg': ['count', 'sum'], 'MehrfachFalsch': ['sum'], 'luecke': ['first'], 'sessionNr':['min'], 'maxSession':['max'], 'vorher_abgebrochen':['max'],  'testpos_prue':['sum'], 'testpos_train':['sum'], 'testpos_vers':['sum']})

def f(row):
    if row.testpos_train['sum'] < 10:
        val = 0
    elif  row.testpos_train['sum'] >= 10:
        if (row.testpos_train['sum'] == 10) and (row.Erfolg['sum'] == 10):
            val = 1
        elif ((row.Erfolg['count']-row.testpos_prue['sum']) == row.Erfolg['sum']) and ((row.testpos_vers['sum']/2+10) ==  row.testpos_train['sum']): #standardweg für Fehler im Training
            val = 1
        elif ((row.Erfolg['count']-row.Erfolg['sum']) == row.testpos_prue['sum']):
            val = 1
        elif ((row.Erfolg['count']-row.Erfolg['sum']+1) == row.testpos_prue['sum']):
            val = 1
        elif (row.testpos_vers['sum']>0) and (row.testpos_prue['sum']==0):
            val = 0
        else:
            val = 3
    else:
        val = 3
    return val

luecke_1_grouped_full['y_all'] = luecke_1_grouped_full.apply(f, axis=1)

<ipython-input-89-d9e585b25a95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  luecke_1['testpos_prue'] = np.where(luecke_1['Testposition']== 'pruefung', 1, 0)
<ipython-input-89-d9e585b25a95>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  luecke_1['testpos_train'] = np.where(luecke_1['Testposition']== 'training', 1, 0)
<ipython-input-89-d9e585b25a95>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [90]:
luecke_1_grouped_full.columns = luecke_1_grouped_full.columns.droplevel(1)
luecke_1_grouped_full = luecke_1_grouped_full.reset_index()
luecke_1_grouped_full = luecke_1_grouped_full[['temp_ueb','y_all']]

luecke_1_grouped.columns = luecke_1_grouped.columns.droplevel(1)
luecke_1_grouped.columns.values[1] = "Erfolg_count"
luecke_1_grouped = luecke_1_grouped.reset_index()

luecke_1_grouped_merged = pd.merge(luecke_1_grouped, luecke_1_grouped_full, on='temp_ueb')

def f(row):
    if row.maxSession == '1' and row.y_all == 1:
        val = 1
    elif row.maxSession == '1' and row.y_all == 0:
        val = 0
    elif row.maxSession != '1':
        if (row.y_all == 1) and (row.maxSession == row.sessionNr) :
            val=1
        else:
            val=0
    else:
        val = 3
    return val

luecke_1_grouped_merged['y'] = luecke_1_grouped_merged.apply(f, axis=1)

In [91]:
luecke_0_grouped.columns = luecke_0_grouped.columns.droplevel(1)
luecke_0_grouped = luecke_0_grouped.reset_index()
luecke_0_grouped = luecke_0_grouped[['UebungsID','y']]
luecke_1_grouped_merged = luecke_1_grouped_merged[['UebungsID','y']]
predictors = luecke_0_grouped.append(luecke_1_grouped_merged)

In [92]:
predictors.to_pickle('predictors_9')